#6. Validação cruzada aninhada

Realizar a validação cruzada aninhada (nested cross-validation) para
encontrar (1) uma boa combinação de hiperparâmetros para ajustar um modelo e (2) um
bom modelo para esse conjunto de dados.

Conjuntos de dados: diamonds.csv

Número de variáveis independentes: 11

Variável dependente: "price"

Modelos utilizados: RandomForest, KNeighbors

##6.1 Importando bibliotecas

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error

##6.2 Importando dados

In [3]:
data_E6 = pd.read_csv("https://raw.githubusercontent.com/RodrigoSFMachado/Data_Science/main/data/diamonds.csv")

##6.3 Pre-processamento de dados

Separando a variável dependente (y) das features (X) nos dados de treino e teste

In [4]:
target_E6 = ['price']
y_E6 = data_E6[target_E6]
X_E6 = data_E6.drop(target_E6, axis=1)

y_final_E6 = y_E6.to_numpy()
y_final_E6 = y_final_E6.ravel()

As variáveis "cut", "color" e "clarity" são classificadas como variáveis categóricas ordinais. Portanto, elas foram categorizadas de acordo com as sequências adequadas de ordem de grandeza.

Realizando o escalonamento dos dados numéricos para mitigar discrepâncias entre as faixas de valores.

In [5]:
standard_labels_E6 = ['carat', 'depth', 'table', 'x', 'y', 'z']

enconder_labels_E6 = {
    'cut' : ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],
    'color' : ['J', 'I', 'H', 'G', 'F', 'E', 'D'],
    'clarity' : ['I1', 'SI2', 'SI1', 'VS2', 'VS1','VVS2', 'VVS1', 'IF']

}

In [6]:
oe_E6 = OrdinalEncoder(categories = list(enconder_labels_E6.values()))
scaler_E6 = StandardScaler()

def data_preparation_E6(X):



  X_encoding_E6 = X[list(enconder_labels_E6)]
  X_encoding_E6 = oe_E6.fit_transform(X_encoding_E6)
  X_encoding_E6 = pd.DataFrame(X_encoding_E6, columns= list(enconder_labels_E6))

  X_standard_E6 = X[standard_labels_E6]
  X_standard_E6 = scaler_E6.fit_transform(X_standard_E6)
  X_standard_E6 = pd.DataFrame(X_standard_E6, columns= standard_labels_E6)

  X_final_E6 = X_encoding_E6.join(X_standard_E6)

  return(X_final_E6)


In [7]:
X_final_E6 =  data_preparation_E6(X_E6)
X_final_E6.head(3)

,cut,color,clarity,carat,depth,table,x,y,z
0,4.0,5.0,1.0,-1.198168,-0.174092,-1.099672,-1.587837,-1.536196,-1.571129
1,3.0,5.0,2.0,-1.240361,-1.360738,1.585529,-1.641325,-1.658774,-1.741175
2,1.0,5.0,4.0,-1.198168,-3.385019,3.375663,-1.498691,-1.457395,-1.741175


##6.4 Criando/treinando/avaliando modelo

In [8]:
models_parameters_E6 = {
    'rfr' : (RandomForestRegressor(),
             {'max_depth' : [10,50,100]}),
    'knnr' : (KNeighborsRegressor(),
              {'n_neighbors' : [5,7,10]})
}

cv_out_E6 = KFold(5)
cv_in_E6 = KFold(3)



Método validação cruzada aninhada:

Treinamento dos modelos de regressão RandomForestRegressor e KneighborsRegressor explorando os hyperparâmetros listados

Avaliação do modelo com melhor resultado através da métrica: MSE(mean-squared-error)

In [9]:
average_out_each_model = dict()

for name, (model, parameters) in models_parameters_E6.items():

  models_bests_hyperparameters = GridSearchCV(
      estimator = model, param_grid= parameters, cv = cv_in_E6, scoring = 'neg_mean_squared_error'
  )

  scores_outer_folds = cross_val_score(
      models_bests_hyperparameters, X_final_E6, y_final_E6, cv = cv_out_E6, scoring = 'neg_mean_squared_error'
  )

  average_out_each_model[name] = np.mean (scores_outer_folds)

  print(f'\n Model: {name} \n MSE in the 5 outer folds: {scores_outer_folds} \n MSE_AVG = {average_out_each_model[name]:.2f} \n')
  print('===================================================================================')




 Model: rfr 
 MSE in the 5 outer folds: [  -653063.43067266  -2177112.6313249  -13940859.90572511
    -61624.94899739   -244483.43886147] 
 MSE_AVG = -3415428.87 


 Model: knnr 
 MSE in the 5 outer folds: [  -841435.97051724  -3399308.58068966 -15713040.6794772
   -114510.42348535   -533628.40952911] 
 MSE_AVG = -4120384.81 



Identificando o hyperparâmetro com o melhor resultado

In [10]:
best_model_name_E6, best_avg_E6 = max(average_out_each_model.items(), key=(lambda avg: avg[1]))
best_model_E6, best_model_parameters_E6 = models_parameters_E6[best_model_name_E6]

final_model_E6 = GridSearchCV(best_model_E6, best_model_parameters_E6, cv = cv_in_E6)
final_model_E6.fit(X_final_E6,y_final_E6)

print(f'\n Best model: {best_model_E6} \n Error (negative mean squared error): {best_avg_E6:.0f} \n Best parameter: {final_model_E6.best_params_}')


 Best model: RandomForestRegressor() 
 Error (negative mean squared error): -3415429 
 Best parameter: {'max_depth': 100}
